In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# --------------------------
# Load UCI HAR dataset
X = pd.read_csv('/kaggle/input/uci-har-dataset/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y = pd.read_csv('/kaggle/input/uci-har-dataset/UCI HAR Dataset/train/y_train.txt', header=None)[0] - 1  # Make labels 0-indexed

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# --------------------------
# Define neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# --------------------------
# Hyperparameters
input_size = X.shape[1]
hidden_size = 64
num_classes = len(np.unique(y))
num_epochs = 30
batch_size = 64
learning_rate = 0.001

# --------------------------
# Create model, loss, optimizer
model = SimpleNN(input_size, hidden_size, num_classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# --------------------------
# Train
model.train()
for epoch in range(num_epochs):
    permutation = torch.randperm(X_train_tensor.size(0))
    for i in range(0, X_train_tensor.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train_tensor[indices].to(device), y_train_tensor[indices].to(device)

        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 5 == 0 or epoch == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# --------------------------
# Evaluate
model.eval()
with torch.no_grad():
    y_pred_logits = model(X_test_tensor.to(device))
    y_pred = torch.argmax(y_pred_logits, dim=1).cpu().numpy()

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

print("\n Test Set Performance:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
import tsfel

In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import tsfel
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# ---------- Load signals ----------
def load_signals(folder, signal_name):
    path = f"/kaggle/input/uci-har-dataset/UCI HAR Dataset/{folder}/Inertial Signals/{signal_name}"
    return np.loadtxt(path)

ax = np.vstack([load_signals('train', 'total_acc_x_train.txt'), load_signals('test', 'total_acc_x_test.txt')])
ay = np.vstack([load_signals('train', 'total_acc_y_train.txt'), load_signals('test', 'total_acc_y_test.txt')])
az = np.vstack([load_signals('train', 'total_acc_z_train.txt'), load_signals('test', 'total_acc_z_test.txt')])

y = np.concatenate([
    np.loadtxt("/kaggle/input/uci-har-dataset/UCI HAR Dataset/train/y_train.txt").astype(int) - 1,
    np.loadtxt("/kaggle/input/uci-har-dataset/UCI HAR Dataset/test/y_test.txt").astype(int) - 1,
])

# ---------- Configure TSFEL ----------
cfg = tsfel.get_features_by_domain()

def extract_features_tsfel(signal_array, fs=50):
    features_list = []
    print(f"Starting TSFEL feature extraction for {len(signal_array)} signals...")
    for idx, row in enumerate(signal_array):
        df_row = pd.DataFrame(row)
        features = tsfel.time_series_features_extractor(cfg, df_row, fs=fs, verbose=0)
        feature_values = np.nan_to_num(features.values[0])
        features_list.append(feature_values)
        if (idx + 1) % 500 == 0 or (idx + 1) == len(signal_array):
            print(f"Extracted features for {idx + 1}/{len(signal_array)} samples")
    return np.array(features_list)

# ---------- Extract features ----------
print("Extracting TSFEL features for ax...")
feat_ax = extract_features_tsfel(ax)
print(" ax done!\n")

print("Extracting TSFEL features for ay...")
feat_ay = extract_features_tsfel(ay)
print(" ay done!\n")

print("Extracting TSFEL features for az...")
feat_az = extract_features_tsfel(az)
print(" az done!\n")

X = np.hstack([feat_ax, feat_ay, feat_az])
print(" All feature extraction complete!")

# ---------- Scale ----------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------- Train/test split ----------
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# ---------- PyTorch NN ----------
class SimpleNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

# ---------- Device setup ----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f" Using device: {device}")

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

# Model, loss, optimizer
input_dim = X_train.shape[1]
num_classes = len(np.unique(y))
model = SimpleNN(input_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
print(" Training started...")
for epoch in range(15):
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 3 == 0:
        _, predicted = torch.max(outputs, 1)
        acc = (predicted == y_train_tensor).float().mean()
        print(f"Epoch [{epoch+1}/15], Loss: {loss.item():.4f}, Train Acc: {acc.item():.4f}")

print(" Training completed.")

# Evaluate
model.eval()
with torch.no_grad():
    outputs_test = model(X_test_tensor)
    _, y_pred_test = torch.max(outputs_test, 1)

y_pred_test_np = y_pred_test.cpu().numpy()
y_test_np = y_test_tensor.cpu().numpy()

print("\n Test Performance:")
print(f"Accuracy: {accuracy_score(y_test_np, y_pred_test_np):.4f}")
print(f"Precision: {precision_score(y_test_np, y_pred_test_np, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_test_np, y_pred_test_np, average='weighted'):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test_np, y_pred_test_np))


In [ ]:
base_path = '/kaggle/input/d/dinesh168/collected-data/Collected data'
activities = ['Laying', 'Sitting', 'Standing', 'Walking', 'Walking_Downstairs', 'Walking_Upstairs']

feature_list = []
true_labels = []
file_names = []

for activity in activities:
    path = os.path.join(base_path, activity)
    for file in os.listdir(path):
        df = pd.read_csv(os.path.join(path, file), header=None)
        df = df.apply(pd.to_numeric, errors='coerce')
        df.dropna(inplace=True)
        df.columns = ['acc_x', 'acc_y', 'acc_z'][:df.shape[1]]

        if len(df) < 128:
            continue
        window = df.iloc[:128]

        tsfel_features_all_axes = []
        for axis in ['acc_x', 'acc_y', 'acc_z']:
            sig_df = pd.DataFrame(window[axis].values)
            tsfel_feat_df = tsfel.time_series_features_extractor(cfg, sig_df, verbose=0)
            tsfel_feat = np.nan_to_num(tsfel_feat_df.values.flatten())
            tsfel_features_all_axes.append(tsfel_feat)

        final_features = np.concatenate(tsfel_features_all_axes)
        feature_list.append(final_features)
        true_labels.append(activity)
        file_names.append(file)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
model = model.to(device)
# Convert
X_new = np.array(feature_list)
X_new_scaled = scaler.transform(X_new)
X_new_tensor = torch.tensor(X_new_scaled, dtype=torch.float32).to(device)

# Predict
with torch.no_grad():
    outputs_new = model(X_new_tensor)
    _, y_pred_new = torch.max(outputs_new, 1)

# Move predictions back to CPU before using numpy()
y_pred_new_cpu = y_pred_new.cpu().numpy()

# Show results
label_map = {
    0: 'Walking',
    1: 'Walking_Upstairs',
    2: 'Walking_Downstairs',
    3: 'Sitting',
    4: 'Standing',
    5: 'Laying'
}

print("\n Predictions on collected data:")
for fname, pred_idx, true_label in zip(file_names, y_pred_new_cpu, true_labels):
    pred_label = label_map.get(pred_idx, "Unknown")
    print(f"{fname}: Predicted = {pred_label}, Actual = {true_label}")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Convert predictions and true labels to numpy arrays
y_pred_new_np = y_pred_new.cpu().numpy()
# Map true activity labels to numeric labels using label_map
true_labels_numeric = []

# Invert label_map for easy lookup
inv_label_map = {v: k for k, v in label_map.items()}

for lbl in true_labels:
    true_labels_numeric.append(inv_label_map.get(lbl, -1))

true_labels_numeric = np.array(true_labels_numeric)

# Compute metrics
acc = accuracy_score(true_labels_numeric, y_pred_new_np)
prec = precision_score(true_labels_numeric, y_pred_new_np, average='weighted', zero_division=0)
rec = recall_score(true_labels_numeric, y_pred_new_np, average='weighted', zero_division=0)
conf_mat = confusion_matrix(true_labels_numeric, y_pred_new_np)

# Print predictions file-wise
print("\n Predictions on collected data:")
for fname, pred_idx, true_lbl in zip(file_names, y_pred_new_np, true_labels):
    pred_label = label_map.get(pred_idx, "Unknown")
    print(f"{fname}: Predicted = {pred_label}, Actual = {true_lbl}")

# Print metrics
print("\n Evaluation Metrics on collected data:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print("Confusion Matrix:\n", conf_mat)


In [ ]:
import os
import pandas as pd
import numpy as np
import tsfel
from sklearn.preprocessing import StandardScaler

base_path = '/kaggle/input/d/dinesh168/collected-data/Collected data'
activities = ['Laying', 'Sitting', 'Standing', 'Walking', 'Walking_Downstairs', 'Walking_Upstairs']

feature_list = []
label_list = []

cfg = tsfel.get_features_by_domain()

label_map = {
    0: 'Walking',
    1: 'Walking_Upstairs',
    2: 'Walking_Downstairs',
    3: 'Sitting',
    4: 'Standing',
    5: 'Laying'
}
inv_label_map = {v: k for k, v in label_map.items()}

for activity in activities:
    path = os.path.join(base_path, activity)
    for file in os.listdir(path):
        df = pd.read_csv(os.path.join(path, file), header=None)
        df = df.apply(pd.to_numeric, errors='coerce')
        df.dropna(inplace=True)
        df.columns = ['acc_x', 'acc_y', 'acc_z'][:df.shape[1]]

        if len(df) < 128:
            continue
        window = df.iloc[:128]

        tsfel_features_all_axes = []
        for axis in ['acc_x', 'acc_y', 'acc_z']:
            sig_df = pd.DataFrame(window[axis].values)
            tsfel_feat_df = tsfel.time_series_features_extractor(cfg, sig_df, verbose=0)
            tsfel_feat = np.nan_to_num(tsfel_feat_df.values.flatten())
            tsfel_features_all_axes.append(tsfel_feat)

        final_features = np.concatenate(tsfel_features_all_axes)
        feature_list.append(final_features)
        label_list.append(inv_label_map.get(activity, -1))

X_collected = np.array(feature_list)
y_collected = np.array(label_list)

# Combine features and labels
X_combined = np.vstack([X_har, X_collected])
y_combined = np.concatenate([y_har, y_collected])

# Re-scale all combined features
scaler_combined = StandardScaler()
X_combined_scaled = scaler_combined.fit_transform(X_combined)

from sklearn.model_selection import train_test_split

# Shuffle and split
X_train, X_test, y_train, y_test = train_test_split(
    X_combined_scaled, y_combined, test_size=0.2, random_state=42, stratify=y_combined
)

import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

input_dim = X_train.shape[1]
num_classes = len(np.unique(y_combined))
model = SimpleNN(input_dim, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(" Training started...")
for epoch in range(15):
    model.train()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 3 == 0:
        _, predicted = torch.max(outputs, 1)
        acc = (predicted == y_train_tensor).float().mean()
        print(f"Epoch [{epoch+1}/15], Loss: {loss.item():.4f}, Train Acc: {acc.item():.4f}")

print(" Training completed.")

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

model.eval()
with torch.no_grad():
    outputs_test = model(X_test_tensor)
    _, y_pred_test = torch.max(outputs_test, 1)

y_pred_test_np = y_pred_test.cpu().numpy()
y_test_np = y_test_tensor.cpu().numpy()

print("\n Combined Test Performance:")
print(f"Accuracy: {accuracy_score(y_test_np, y_pred_test_np):.4f}")
print(f"Precision: {precision_score(y_test_np, y_pred_test_np, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_test_np, y_pred_test_np, average='weighted'):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test_np, y_pred_test_np))
